In [91]:
import sys
import os
sys.path.append("../../")
import pandas as pd
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt
import numpy as np

In [68]:
results = {}
for n_sats in range(3,13,1):
    results[n_sats] = {'Optimization': {}, 'Planning': {}}

    if os.path.exists(f'../../Pop_Over_Gen/{n_sats}.csv'):
        df = pd.read_csv(f'../../Pop_Over_Gen/{n_sats}.csv')
        results[n_sats]['Pop_Over_Gen'] = df

    for n_targets in [15,34,65,82,109,186,494]:
        if os.path.exists(f'../../Optimization_Results/{n_sats}_{n_targets}.csv'):
            df = pd.read_csv(f'../../Optimization_Results/{n_sats}_{n_targets}.csv')
            results[n_sats]['Optimization'][n_targets] = df

        if os.path.exists(f'../../Planning_Results/{n_sats}_{n_targets}.csv'):
            df = pd.read_csv(f'../../Planning_Results/{n_sats}_{n_targets}.csv')
            results[n_sats]['Planning'][n_targets] = df

In [183]:
def pop_over_gen_ploter(n_sats, results):
    fig = make_subplots()
    colors = ['blue','orange','green','red','purple','brown','pink','gray','cyan','olive']
    frames = []

    for i, gen in enumerate(results[n_sats]['Pop_Over_Gen']['Gen'].unique()):
        df = results[n_sats]['Pop_Over_Gen'][results[n_sats]['Pop_Over_Gen']['Gen'] == gen]

        scatter = go.Scatter(
            x=df['Avg_Time'],
            y=df['Avg_Percentage'],
            # hovertext=df[dvs_ovs].astype(str).agg(', '.join, axis=1),
            # hoverinfo='text',
            mode='markers',
            name=f'{gen}',
            marker=dict(color=colors[i], size=8),)

        fig.add_trace(scatter)

        frames.append(go.Frame(data=[scatter], name=str(gen)))

    fig.add_vline(30,line_dash = 'dash', line_color='red')
    fig.add_hline(100,line_dash = 'dash', line_color='red')


    fig.add_annotation(
        x=30.5,  # X position for vertical line annotation
        y=50,  # Adjust this for vertical annotation (above the line)
        text='30 Day Constraint',
        font=dict(color='red', size=15),
        textangle=90,
        showarrow=False
        )

    fig.add_annotation(
        x=15,
        y=105,
        text='100% Imaged Constraint',
        font=dict(color='red', size=15),
        showarrow=False)


    fig.update_layout(
        title=f'{n_sats} Satellites Pop_Over_Gen',
        xaxis_title='Average Time to Image [days] ←',
        yaxis_title='Average Target Bin Percentage →',
        legend=dict(
            title='Gen',
            title_font=dict(size=15),
            font=dict(size=12, color='black'),
            xanchor='right',
            yanchor='bottom',
        ),
        template='plotly_white',
        font=dict(size=15),
        updatemenus=[
            {
                'buttons': [
                    {
                        'args': [None, {'frame': {'duration': 500, 'redraw': True}, 'mode': 'immediate'}],
                        'label': 'Play',
                        'method': 'animate'
                    },
                    {
                        'args': [[None], {'frame': {'duration': 0, 'redraw': True}, 'mode': 'immediate'}],
                        'label': 'Pause',
                        'method': 'animate'
                    }
                ],
                'direction': 'left',
                'pad': {'r': 10, 't': 87},
                'showactive': False,
                'type': 'buttons',
                'x': 0.1,
                'xanchor': 'right',
                'y': 1.1,
                'yanchor': 'top'
            }
        ],
        sliders=[{
            'active': 0,
            'yanchor': 'top',
            'xanchor': 'left',
            'currentvalue': {
                'prefix': 'Gen: ',
                'visible': True,
                'xanchor': 'right'
            },
            'transition': {'duration': 300, 'easing': 'cubic-in-out'},
            'pad': {'b': 10},
            'len': 0.9,
            'x': 0.1,
            'y': 0,
            'steps': [{
                'args': [
                    [gen],
                    {'frame': {'duration': 500, 'redraw': True}, 'mode': 'immediate'}
                ],
                'label': str(gen),
                'method': 'animate'
            } for gen in results[n_sats]['Pop_Over_Gen']['Gen'].unique()]
        }]
    )

    # Add frames to the figure
    fig.frames = frames

    fig.show()

pop_over_gen_ploter(8, results)